In [1]:
import nest

nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

nest.projections.Connect([nest.projections.OneToOne(source=n, target=n),
                          nest.projections.OneToOne(source=n, target=n)])
nest.projections.BuildNetwork()


[[NodeCollection(metadata=None, model=iaf_psc_alpha, size=5, first=1, last=5), NodeCollection(metadata=None, model=iaf_psc_alpha, size=5, first=1, last=5), {'rule': 'one_to_one'}, {'synapse_model': 'static_synapse'}], [NodeCollection(metadata=None, model=iaf_psc_alpha, size=5, first=1, last=5), NodeCollection(metadata=None, model=iaf_psc_alpha, size=5, first=1, last=5), {'rule': 'one_to_one'}, {'synapse_model': 'static_synapse'}]]


## Projection object

Introducing the Projection object, representing projection between two populations with a certain rule. The projection object is added to a buffer with `projections.Connect()`, and held until calling `projections.BuildNetwork()`.

### `OneToOne`

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.OneToOne(source=n, target=n)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### `FixedIndegree`
Different Projections can take different arguments, depending on the connection rule.

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedIndegree(source=n, target=n, indegree=5)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### `FixedOutdegree`

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedOutdegree(source=n, target=n, outdegree=5)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### `PairwiseBernoulli`
Projections can store synapse specifications.

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 10)

projection = nest.projections.PairwiseBernoulli(source=n, target=n, 
                                                p=0.2)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### `FixedTotalNumber`
Can also pass arguments that controls if autapses or multapses are possible.

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

projection = nest.projections.FixedTotalNumber(source=n, target=n, N=10, 
                                               allow_autapses=True,
                                               allow_multapses=True)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

## Multiple projections
Multiple Projections can be added to the buffer before connecting by calling `projections.Connect()` multiple times.

In [ ]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A))
nest.projections.Connect(nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B))
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

Or by passing a list of Projections.

In [ ]:
nest.ResetKernel()

N = 10
IN_A = 2
IN_B = 5
n = nest.Create('iaf_psc_alpha', N)
projections = [nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A),
               nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B)]
nest.projections.Connect(projections)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

## Using `ConnectImmediately()`
Using `projections.ConnectImmediately()` will immediately construct the connections of a Projection.

In [ ]:
nest.ResetKernel()

n = nest.Create('iaf_psc_alpha', 5)
nest.projections.ConnectImmediately(nest.projections.OneToOne(source=n, target=n))

conns = nest.GetConnections()
print(conns)


Or for all Projections in a list.

In [ ]:
nest.ResetKernel()

N = 5
IN_A = 2
IN_B = 4
n = nest.Create('iaf_psc_alpha', N)

projections = [nest.projections.FixedIndegree(source=n, target=n, indegree=IN_A),
               nest.projections.FixedIndegree(source=n, target=n, indegree=IN_B)]
nest.projections.ConnectImmediately(projections)

conns = nest.GetConnections()
print(conns)

## Synapse model object
### `static_synapse`

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)

projection = nest.projections.OneToOne(source=n, target=n, syn_spec=synapse)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### `stdp_synapse`

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 5)

synapse = nest.synapsemodels.stdp(weight=nest.random.uniform(0.5, 1.0), tau_plus=17.)

projection = nest.projections.OneToOne(source=n, target=n, syn_spec=synapse)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('tau_plus =', conns.tau_plus)

## Usage examples

### Generate projections iteratively, with duplicated projections
By cloning a Projection, one can iteratively generate similar projections, where only a few parameters are changing.

In [ ]:
nest.ResetKernel()
n1 = nest.Create('iaf_psc_alpha', 3)
n2 = nest.Create('iaf_psc_exp', 3)

synapse = nest.synapsemodels.static(weight=0.5, delay=0.7)
base_projection = nest.projections.FixedTotalNumber(source=n1, target=n2,
                                                    N=5,
                                                    syn_spec=synapse)
for s, t in ((n1, n2), (n1, n1), (n2, n2), (n2, n1)):
    p = base_projection.clone()  # Create a copy of the base Projection.
    p.source = s  # Change some parameters
    p.target = t  # of the copied Projection.
    nest.projections.Connect(p)  # Add the copy to the buffer.
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)
print('Number of connections:', len(conns))

### Using `CollocatedSynapses` in projections

In [ ]:
nest.ResetKernel()
n = nest.Create('iaf_psc_alpha', 2)

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.projections.FixedIndegree(source=n, target=n, indegree=3, syn_spec=syn_spec)
nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### Spatial connections

In [ ]:
nest.ResetKernel()

dim = [4, 5]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}
projection = nest.projections.FixedIndegree(source=layer, target=layer, indegree=1, mask=mask)

nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)

### Spatial connections with `CollocatedSynapses`

In [ ]:
nest.ResetKernel()

dim = [4, 2]
extent = [10., 10.]
layer = nest.Create('iaf_psc_alpha', positions=nest.spatial.grid(dim, extent=extent))

mask = {'rectangular': {
        'lower_left': [-5., -5.],
        'upper_right': [0., 0.]}}

syn_spec = nest.CollocatedSynapses(nest.synapsemodels.static(weight=-2.),
                                   nest.synapsemodels.stdp(weight=3.))

projection = nest.projections.FixedIndegree(source=layer, target=layer, indegree=1, syn_spec=syn_spec, mask=mask)

nest.projections.Connect(projection)
nest.projections.BuildNetwork()

conns = nest.GetConnections()
print(conns)